## Adding simple db to store lastest timestamp

I evision the following simple db with one domain <br>
Domain Name: awaybot <br>
For each team, we'll create an item that is the team id (from slack api) <br>
For each item, we'll have three attributes <br>
1. Team name
2. Channel name: the channel that we last read a message from
    1. We'll set the replace value to true, as we'll update this for each message
3. ts: The timestamp of the last message sent to kafka,
    1. We'll set the replace value to true, as we'll update this for each message

Whenever we query for the latest timestamp, we'll use `ConsistentRead=True` to ensure we don't get a stale timestamp


#### System Reqs

1. boto3
    1. pip install boto3

In [23]:
import boto3
# Connect to simpleDB
client = boto3.client('sdb')
# Create a test domain (table in RDMS terms)
client.create_domain(
    DomainName='test')

# Create a test item and put attributes into it
# Note that values are always strings
test_item_name = 'U3G5603L'
item_attrs = [
    {'Name': 'Team', 'Value': 'U3G5603L'},
    {'Name': 'Channel', 'Value': 'U5LMN360', 'Replace': True},
    {'Name': 'ts', 'Value': '174435541.5', 'Replace': True}
    ]
response = client.put_attributes(
    DomainName='test',
    ItemName=test_item_name,
    Attributes=item_attrs
)

In [24]:
# Read back the attributes using consistent read, so that we don't get stale reads
response = client.get_attributes(
    DomainName='test',
    ItemName='U3G5603L',
    AttributeNames=[
        'ts',
        'Channel'
    ],
    ConsistentRead=True
)

response['Attributes']

[{u'Name': 'Channel', u'Value': 'U5LMN360'},
 {u'Name': 'ts', u'Value': '174435541.5'}]

In [25]:
# Delete test domain omain
response = client.delete_domain(
    DomainName='string'
)